## Etude des modes jeu et statistiques

### Partie graphique du mode jeu

**Q1.** L'image se trouve dans le répertoire "C:/CCP" et s'appelle "stade.bmp". Il suffit dès lors d'utiliser les rappels de l'annexe 1.

In [ ]:
# Solution

import os

#L'image se trouve dans le répertoire "C:/CCP"
os.chdir('C:/CCP')

import scipy.misc as scm

# L'image s'appelle "stade.bmp" et doit être chargée dans une variable nommée image_terrain
image_terrain = scm.imread("stade.bpm")
scm.imshow(image_terrain)

* *la fonction `imread` est obslète à partir de la version 1.2.0 de `scipy`. Il faut utiliser la fonction `imageio.imread` du module `imageio` comme indiqué dans l'aide de la fonction accessible via la commande `help(scm.read)`.*
* *Il y a une erreur dans le code de l'annexe ligne 7, il faut lire `scm.imshow(picture)`.*
* *Une image est un exemple de tableau numpy (de type `ndarray`) de dimension 3.*

**Q2.** La longeur d'une image correspond au nombre de colonnes i.e. à la deuxième valeur renvoyée par la commande `shape` rappelée en annexe. La largeur correspond elle au nombre de lignes, donc à la première valeur. Pour afficher le résultat, il faut utiliser le formatage des chaînes de caractères, rappelé dans l'annexe 1.

In [ ]:
# dimensions de l'image
dim_long = image_terrain.shape[1]
dim_larg = image_terrain.shape[0]

# formatage d'une chaine de caractères
print("%d x %d"%(dim_long,dim_larg))

* *L'emplacement réservé `%d` est utilisé pour insérer un entier. Il existe aussi :*
    - `%s` *pour insérer une chaine de caractères,*
    - `%f` *pour insérer un flottant.*
* *Il est préférable d'utiliser la méthode `.format` des chaines de caractères :*
    
        "{} x {}".format(dim_long,dim_larg)

**Q3.** Cette question est très vague : on ne sait pas vraiment s'il faut éviter les lignes, on ne connait pas les dimensions de l'image et surtout, une fonction ne renvoit pas une variable... (elle pourrait éventuellement modifier une variable globale nommée `coul_ter`). Voici l'extrait du rapport du jury : 

*"Il s’agissait ici de vérifier que les candidats étaient capables de récupérer une valeur dans un tableau
à partir d’indices calculés. Beaucoup oublient que ces indices doivent être entiers."*

On se contente donc de la fonction suivante, la plus "naturelle" :

In [ ]:
# Les dimensions de l'image doivent être stockées dans les variables dim_long et dim_larg

def coul(image):
    
    # Centre (entier) du terrain
    centre = dim_long//2, dim_larg//2
    
    return image[centre]

# Stockage de la couleur du terrain dans la variable coul_ter
coul_ter = coul(image_terrain)

*Les commandes `a//b` et `a%b` renvoient respectivement le quotient et le reste de la division euclidienne de `a:int` par `b:int`. Ce sont deux entiers.*  

**Q4.** Conformément aux rappels de l'annexe 1, la variable `coul_ter` est un triplet. *En fait, la terminologie utilisée par l'annexe 1 n'est pas la bonne car il s'agit en fait de tableau numpy et non de tuples.*  Il faut un octet ($8$ bits) pour stocker un entier entre $0$ et $255$. Pusiqu'on doit stocker un triplet de tels entiers, il faut $3$ octets i.e. 24 bits.

**Q5.** La encore, cette fonction est très imprécise : doit on stocker le blanc de l'image, le blanc absolu correspondant au triplet $[255,255,255]$ ?... On fera donc au plus simple. Le rapport du jury précise : *"l'objectif était de réutiliser la fonction précédente et le tableau image"*.

In [ ]:
def maillot(image):
    
    # Couleur du terrain
    coul_ter = coul(image)
    
    # blanc
    blanc = [255,255,255]
    
    return [coul_ter, blanc]

### Partie graphique du mode statistique

**Q6.** Ici encore cette question n'est pas assez précise. Par exemple, si $A$ est de taille $4$, on ne sait pas quelle matrice prendre pour entourer $b_{ij}$. Ceci est vrai dès que $A$ est de taille paire. On va donc considérer que $A$ est de taille *impaire*. Les questions **Q6.** et **Q7.** nécessitent d'utiliser le *slicing multi-dimensionnel* qui permet d'extraire une partie d'un tableau. La syntaxe générale du slicing est : 

    L[debut:fin:pas] # Extraire de L les termes d'indice "debut" (inclu) à "fin" (exclu) avec un pas de "pas".

où `début` et `fin` peuvent prendre des valeurs vides (remplacées par $0$), tandis que `:pas` est optionnel.

In [24]:
# On suppose que la bibliotèque numpy a été importée et surnomée np

def filtrer1(filtreA,matB):
    nA = filtreA.shape[0]
    nB_ligneB = matB.shape[0] # Règle du licol
    nB_colonneB = matB.shape[1]
    C = matB.copy() # initialisation de la matrice C (les colonnes et lignes non modifiées seront celles de B)
    bordure = nA//2 # Il ne faut pas modifier les nA//2 lignes et colonnes du bord
    for i in range(bordure,nB_ligneB-bordure):
        for j in range(bordure,nB_colonneB-bordure):
            Bij = matB[i-bordure:i+bordure+1,j-bordure:j+bordure+1]# slicing multi-dimensionnel
            C[i,j] = np.sum(Bij.dot(A)) 
    return C

**Q7.** On va utiliser la question précédente pour traiter les trois matrices constituant une image.

In [ ]:
def filtrer(filtreA,matB):
    C = matB.copy() # C a la même taille que B
    
    # Mise a jour de C
    for i in range(3):
        C[:,:,i] = filtrer1(filtreA,matB[:,:,i])
    return C

* *si B est un tableau numpy de dimension 3, de taille $n\times p\times 3$,* `B[:,:,i]` *est un tableau numpy de dimension 2 (une matrice), de taile $n\times p$.*
* *il a une erreur dans la méthode proposée. Pour moyenner sur les pixels voisins, il ne faut pas prendre la somme des coefficients de $C$, mais la somme des coefficients diagonaux de $C$ i.e. la trace de $C$ car $\sum b_{ij}a_{ij} = Tr(A^TB)$.*

**Q8.** Pour cette question, il faut faire un dessin avec les valeurs de $x$ et $y$ sur l'exemple donné.

In [116]:
def matriceFlouGaussien(taille, sigma):
    """
    taille : taille de la matrice (impaire)
    sigma : écart type (déviation standard)
    retourne un niveau gaussien 
    """
    mat = np.zeros([taille,taille]) # usuellement l'argument de np.zeros est plutôt un tuples...
    taille = taille//2
    for x in range(-taille, taille+1):
        for y in range(-taille, taille + 1):
            mat[x + taille,y + taille] = 1/(sigma*np.sqrt(2*np.pi))*np.exp(-(x**2 + y**2)/(2*sigma**2))                                                                       
    return mat/(np.sum(mat)) # Normalisation

**Q9.** Il suffit d'appliquer les fonctions précédement construites :

In [ ]:
def FloutageGaussien(tabPix, taille, sigma):
    
    # Construction du filtre
    filtreA = matriceFlouGaussien(taille, sigma)
    
    # Filtrage
    return filtrer(filtreA,tabPix)

### Fonctionnalités du mode statistique

**Q10**. C'est une simple question de *slicing* puisqu'il s'agit d'extraire des colonnes de la matrice *résultat*.

In [ ]:
# x
x = resultat[:,0] 

#y_plaR
y_plaR = resultat[:,3]

**Q11.** Ici encore, on ne comprend pas vraiment ce que demande l'énoncé... Pour que la commande de l'exemple fnctionne, on a `concaténer` les différentes valeurs pour construire les différentes listes. Puisque les valeurs extrèmes sont incluses, on va utiliser la commande `linspace` du module `numpy`.

In [105]:
# Listes pour l'histogramme

# initialisation
barre = []
abscisse = []

for i in range(len(x)):
    num = int((y_plaR[i] - 0)/0.1) + 1 # Faire un dessin
    valeurs_barre = list(np.linspace(0,y_plaR[i],num)) # La commande '+' permet de concaténer des listes,
                                                       # pas des tableaux numpy.
    valeurs_abscisse = list(x[i]*np.ones(len(valeurs_barre)))
    barre = barre + valeurs_barre
    abscisse = abscisse + valeurs_abscisse

**Q12** Il s'agit d'une question classique d'algorithme. Il est sous-entendu ici que l'on s'interdira d'utiliser les fonctions existantes `min`, `max` et `sum`. 

In [112]:
def minMaxMoy(valeurs):
    
    #initialisation
    Min, Max, s = valeurs[0], valeurs[0], 0
    
    for x in valeurs :
        if x < Min:
            Min = x
        if x > Max:
            Max = x
        s = s + x
    
    # Calcule de la moyenne
    Moy = s/len(valeurs)
    
    return Min, Max, Moy

## Traitement du transfert des joueurs

**Q13.** Il faut bien connaître les [*opérateurs de comparaison*](https://sql.sh/cours/where) en SQL que l'on utilise avec le mot clé `WHERE`.

    SELECT Nom FROM Joueurs WHERE VMA > 13 AND Age > 23

*Il n'est pas très rigoureux de la part de l'énoncé de ne pas préciser le type des attributs des tables.*

**Q14.** Les informations nécessaires sont présentes dans plusieurs tables. Il va donc falloir effectuer une jointure. Ici encore, l'énoncé manque de rigueur car aucune clé primaire ni étrangère ne sont précisées.  

    SELECT Clubs.Nom 
    FROM Clubs 
        JOIN Joueurs ON Clubs.Id_Club = Joueurs.Id_Club
    WHERE Salaire euros > 30 000
    
On va ensuite compter un nombre, donc utiliser la fonction d'agrgégation `COUNT`.

    SELECT COUNT(Joueurs.Nom)
    FROM Joueurs
        JOIN Clubs ON Joueurs.Id_Club = Clubs.Id_Club
    WHERE Clubs.Nom = 'Stade Toulousain' AND Poste = 'Talonneur'

**Q15** On va de même utiliser la fonction d'agrégation `SUM` et donc un `GROUP BY` (par nom de club).

        SELECT Clubs.Nom, SUM(Salaire euros)/Budget total * 100
        FROM Clubs 
            JOINS Joueurs ON Clubs.Id_Club = Joueurs.Id_Club
        GROUP BY Clubs.Nom

## Evaluation des performances de l'équipe

**Q16.** Il est impossible de donner la réponse à cette question sans disposer de l'aide de la métode [`fetchall`](https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-fetchall.html) ! Peut être s'agit-il d'un type propre à l'extension `sqlite3`, comme les tableaux de type `ndarray` du module `numpy`... Ici, la réponse attendue était une "liste". Le rapport précise que l'on pouvait deviner cette réponse en lisant par exemple le code de la question **Q17**... Le type du résultat effectivement renvoyé par `fetchall` est une *liste (de tuples).*

**Q17.** Il faut remplacer la ligne 7 par `for i in range(len(liste))` et remplacer le `>` de la ligne 10 par un `<`. La question est mal écrite car on a l'impression qu'il faut aussi modifier l'appel de la fonction, qui est correct. 

**Q18.** Notons $n$ la taille de la liste à trier. Il y a deux boucles `for` imbriquées. La première boucle est parcouru $n$ fois, la seconde boucle est parcouru $n-1$, puis $n-2$,..., puis $1$ fois.   Dans le pire des cas (liste rangée dans l'ordre décroissant), l'appel à la fonction `echange` est systèmatique dans la deuxième boucle. Cette fonction a une complexité *constante* (indépendante de $n$). Au final, la complexité est donc en :
$$
n\times\left(\sum_{k=1}^{n-1} O(1)\right) = n\times \frac{n(n-1)}{2}\times O(1) =O(n^2).
$$
Il s'agt donc d'une complexité *quadratique*. C'est une complexité assez importante, qui ne permet pas de manipuler des données trop grandes (cf https://fr.wikipedia.org/wiki/Analyse_de_la_complexité_des_algorithmes ), mais qui n'est pas du tout rédibitoire pour classer une équipe de foot.  

**Q19.** Il s'agit de l'algotithme de [tri rapide](https://fr.wikipedia.org/wiki/Tri_rapide).  
* la ligne 19 détecte si $i=j$ i.e. si le pivot $k$ a atteint une des extrémités. Dans ce cas, le sous-tableau à trier possède un seul élément, il est donc trié,  
* la ligne 20 fait appelle à la fonction `segmente` qui place l'élément $i$ à sa place définitive et définit le nouveau pivot $k$,
* les lignes 21 et 22 trient les deux sous-tableaux situés à droite et à gauche du pivot,
* la ligne 23 renvoie la liste triée.  
La fonction `tri_2` est une fonction *récursive*. Pour compter le nombre d'appels récursifs, il faut incrémenter un compteur à chaque appel de la fonction. Cependant, pour garder trace de ce compteur hors de la fonction (lors des appels succesifs), il faut que cette variable soit déclarée comme `globale` :

In [2]:
# Variable globale
global compteur

# Initialistion
compteur = 0

def tri_2(L,val,i,j):
    compteur += 1 # Incrémentation du compteur
    if i<j:
        k = segmente(L,val,i,j)
        tri_2(L,val,i,k-1)
        tri_2(L,val,k+1,j)
    return compteur

0


**Q20.** Si on veut trier en fonction du poids des joueurs, alors `val = 3` car le poids correspond à la 4ieme colonne (indide n°3). On utilise donc le code suivant :

    tri_2(monequipe,3,0,len(monequipe))

**Q21.** On a $74 =(1001010)_2$ et $0.25 = (0.01)_2$ donc $74.25 = (1001010.01)_2$.  

**Q22.**  Pour des rappels sur la représentation des réels en virgule flottante et la norme IEEE754, voir https://fr.wikipedia.org/wiki/IEEE_754. On a $74.25 = (1001010.01)_2 = 1.00101001\times 2^6$. C'est un nombre flottant normalisée : la mantisse se lit à droite de la virgule (complétée par des $0$), elle vaut $001 0100 1000 0000 0000$. L'exposant décalé vaut $6+127 = 133 = (10000101)_2$. Puisque $74.25$ est positif, la représentation de $74.25$ suivant la norme IEEE 754 32 bits est : 
$$
74.25 = 0\ 10000101\ 001\ 0100\ 1000\ 0000\ 0000
$$

Il y a trois caractériqtiques à stocker par jouers, donc $3000\times 3$ flottants de $32$ bits. Ceci représente environ $288$ $ko$ de données. Le format simple précision est *moins gourmand en espace de stockage* que le double précision. Par contre il est *moins précis* ($7$ chiffres significatifs contre $16$).

 ## Entrainement à la passe vissée au rugby

**Q24.** Ici encore, l'idée semble être d'éviter de se servire des fonctions du module numpy qui sont parfaitement adaptées pour ce type de tâches... Encore une fois, cette question est ambigüe. Par exemple, on ne sait pas si `tmax` doit être inclu dans la liste renvoyée comme le fait la fonction `linspace` du module `numpy`, ou exclu, comme la fonction `arange`. Puisque cette dernière est défini à l'aide d'un pas, nous allons la copier, et à priori exclure `tmax` (sauf si c'est un multiple entier de pas). 

In [5]:
def liste_temps(pas, temps):
    L = []
    for i in range(int(temps/pas)+1): # Attention, l'argument de la fonction int() doit être un entier
        L.append(i* pas)
    return L

**Q25.** Il s'agit d'une équation différentielle linéaire du première ordre, non-homogène. En remarquant que $v = K_cU_0$ est une solution particulière, on a d'après le cours : 
$$
\boxed{\forall t\geqslant 0,\ v(t) = K_cU_0(1-e^{-\frac{t}{\tau}}).}
$$
La suite de la question peut alors se traiter classiquement, à l'aide d'une compréhension de liste :

In [10]:
def vitesse(k,tau,u,temps):
    return [k*u*(1-np.exp(t/tau)) for t in temps]

**Q26.** Bizarement, les notations changent d'une question à l'autre : $u$ est maintenant appelé $U_0$ et $k$ devient $K_c$. De plus, *temps* est maintenant une liste alors que c'était un réel à la question **Q24.** ... Quoi qu'il en soit, il faut parfaitement connaître et savoir mettre en oeuvre la méthode d'Euler. 

In [22]:
def ordre1_euler(Kc, tau, U0, temps):
    
    # Définition du pas de temps et de la vitesse initiale
    pas = temps[1]-temps[0]
    vitesse =[0]
    
    # Algorithme d'Euler
    for t in temps[1:]: # On connait la vitesse à t = 0
        vitesse_suivante = pas*(1/tau*(Kc*U0 - vitesse[-1])) + vitesse[-1] # Règle d'accès inversé
        vitesse.append(vitesse_suivante)
        
    return vitesse

**Q27.** On va utiliser trois fois les fonctions précédentes dans une boucle, et stoker les $3$ résultats (de temps et de vitesse) dans deux listes `temps` et  `vitesse`.

In [13]:
liste_pas = [0.2,0.4,0.6]
vitesse = []
temps = []

for pas in liste_pas :
    temps.append(liste_temps(pas,temps))
    vitesse.append(ordre1_euler(Kc,tau,U0,temps))

NameError: name 'liste_temps' is not defined

In [ ]:
#importation du module graphique de python
import matplotlib.pyplot as plt

#temps
for i in range(3):
    plt.plot(temps[i],vitesse[i], label = 'pas de {}'.format(liste_pas[i]))

#affichage
plt.xlabel('t')
plt.ylabel('v(t)')
plt.legend()
plt.show()